In [50]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score

In [16]:
#import db
db=pd.read_csv("student-mat.csv")
print(db.shape)
db.head()

(395, 33)


,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10


In [17]:
#G3 stünununu 10 üzeri puanları başarılı(1), altında kalan puanları başarısız(0) olacak şekilde düzenliyoruz
df=db.copy()
df['PASS']=df['G3'].map(lambda x:0 if x<10 else 1)
df.drop(['G3'],axis=1,inplace=True)
df

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,PASS
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,0
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,0
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,1
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,1
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,MS,M,20,U,LE3,A,2,2,services,services,...,5,5,4,4,5,4,11,9,9,0
391,MS,M,17,U,LE3,T,3,1,services,services,...,2,4,5,3,4,2,3,14,16,1
392,MS,M,21,R,GT3,T,1,1,other,other,...,5,5,3,3,3,3,3,10,8,0
393,MS,M,18,R,LE3,T,3,2,services,other,...,4,4,1,3,4,5,0,11,12,1


In [18]:
#veri sayısallaştırma (https://medium.com/datarunner/veri-%C3%B6n-i%CC%87%C5%9Fleme-kategorik-verilerin-say%C4%B1salla%C5%9Ft%C4%B1r%C4%B1lmas%C4%B1-6ba7e78c1be1)
from sklearn.preprocessing import LabelEncoder
#binary
temp=df.iloc[:,0:1].values
temp[:,0]=LabelEncoder().fit_transform(temp[:,0])
df["school"]=temp
temp=df.iloc[:,1:2].values
temp[:,0]=LabelEncoder().fit_transform(temp[:,0])
df["sex"]=temp
temp=df.iloc[:,3:4].values
temp[:,0]=LabelEncoder().fit_transform(temp[:,0])
df["address"]=temp
temp=df.iloc[:,4:5].values
temp[:,0]=LabelEncoder().fit_transform(temp[:,0])
df["famsize"]=temp
temp=df.iloc[:,5:6].values
temp[:,0]=LabelEncoder().fit_transform(temp[:,0])
df["Pstatus"]=temp
temp=df.iloc[:,15:16].values
temp[:,0]=LabelEncoder().fit_transform(temp[:,0])
df["schoolsup"]=temp
temp=df.iloc[:,16:17].values
temp[:,0]=LabelEncoder().fit_transform(temp[:,0])
df["famsup"]=temp
temp=df.iloc[:,17:18].values
temp[:,0]=LabelEncoder().fit_transform(temp[:,0])
df["paid"]=temp
temp=df.iloc[:,18:19].values
temp[:,0]=LabelEncoder().fit_transform(temp[:,0])
df["activities"]=temp
temp=df.iloc[:,19:20].values
temp[:,0]=LabelEncoder().fit_transform(temp[:,0])
df["nursery"]=temp
temp=df.iloc[:,20:21].values
temp[:,0]=LabelEncoder().fit_transform(temp[:,0])
df["higher"]=temp
temp=df.iloc[:,21:22].values
temp[:,0]=LabelEncoder().fit_transform(temp[:,0])
df["internet"]=temp
temp=df.iloc[:,22:23].values
temp[:,0]=LabelEncoder().fit_transform(temp[:,0])
df["romantic"]=temp
#numeric

Mjob = pd.get_dummies(df["Mjob"])
Fjob = pd.get_dummies(df["Fjob"])
reason = pd.get_dummies(df["reason"])
guardian = pd.get_dummies(df["guardian"])
Mjob = Mjob.rename(columns={"teacher":"Mother_teacher","at_home": "Mother_at_home", "health" : "Mother_health", "services":"Mother_services","other":"Mother_other"})
Fjob = Fjob.rename(columns={"teacher":"Father_teacher","at_home": "Father_at_home", "health" : "Father_health", "services":"Fther_services","other":"Father_other"})
guardian = guardian.rename(columns={"other":"g_other"})
df=pd.concat([df,Mjob,Fjob,reason,guardian],join='inner',axis=1)
df.drop(["Mjob","Fjob","reason","guardian"],axis=1,inplace=True)
df


,school,sex,age,address,famsize,Pstatus,Medu,Fedu,traveltime,studytime,...,Father_other,Fther_services,Father_teacher,course,home,other,reputation,father,mother,g_other
0,0,0,18,1,0,0,4,4,2,2,...,0,0,1,1,0,0,0,0,1,0
1,0,0,17,1,0,1,1,1,1,2,...,1,0,0,1,0,0,0,1,0,0
2,0,0,15,1,1,1,1,1,1,2,...,1,0,0,0,0,1,0,0,1,0
3,0,0,15,1,0,1,4,2,1,3,...,0,1,0,0,1,0,0,0,1,0
4,0,0,16,1,0,1,3,3,1,2,...,1,0,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,1,1,20,1,1,0,2,2,1,2,...,0,1,0,1,0,0,0,0,0,1
391,1,1,17,1,1,1,3,1,2,1,...,0,1,0,1,0,0,0,0,1,0
392,1,1,21,0,0,1,1,1,1,1,...,1,0,0,1,0,0,0,0,0,1
393,1,1,18,0,1,1,3,2,3,1,...,1,0,0,1,0,0,0,0,1,0


In [20]:
df.groupby("PASS")["PASS"].count()

PASS
0    130
1    265
Name: PASS, dtype: int64

In [21]:
#özellik ve etiketlerin ayrılması
x=df.drop("PASS",axis=1)
y=df["PASS"]

In [22]:
x.columns

Index(['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu',
       'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid',
       'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel',
       'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences', 'G1', 'G2',
       'Mother_at_home', 'Mother_health', 'Mother_other', 'Mother_services',
       'Mother_teacher', 'Father_at_home', 'Father_health', 'Father_other',
       'Fther_services', 'Father_teacher', 'course', 'home', 'other',
       'reputation', 'father', 'mother', 'g_other'],
      dtype='object')

In [23]:
#SMOTE---https://towardsdatascience.com/applying-smote-for-class-imbalance-with-just-a-few-lines-of-code-python-cdf603e58688
from imblearn.over_sampling import SMOTE
smote = SMOTE()
X_res, y_res = smote.fit_resample(x, y)

In [24]:
X_res


,school,sex,age,address,famsize,Pstatus,Medu,Fedu,traveltime,studytime,...,Father_other,Fther_services,Father_teacher,course,home,other,reputation,father,mother,g_other
0,0.0,0.0,18,1.0,0.0,0.0,4,4,2,2,...,0,0,1,1,0,0,0,0,1,0
1,0.0,0.0,17,1.0,0.0,1.0,1,1,1,2,...,1,0,0,1,0,0,0,1,0,0
2,0.0,0.0,15,1.0,1.0,1.0,1,1,1,2,...,1,0,0,0,0,1,0,0,1,0
3,0.0,0.0,15,1.0,0.0,1.0,4,2,1,3,...,0,1,0,0,1,0,0,0,1,0
4,0.0,0.0,16,1.0,0.0,1.0,3,3,1,2,...,1,0,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525,0.0,0.0,16,1.0,0.0,1.0,2,2,1,2,...,1,0,0,0,1,0,0,0,1,0
526,0.0,1.0,17,0.0,0.403986,1.0,2,1,1,1,...,1,0,0,0,0,0,0,0,1,0
527,0.0,0.183824,16,1.0,0.183824,1.0,4,3,1,2,...,0,0,0,0,0,0,0,0,1,0
528,0.0,0.809305,21,1.0,0.0,1.0,3,1,1,1,...,0,1,0,0,0,0,0,0,0,0


In [25]:
y_res

0      0
1      0
2      1
3      1
4      1
      ..
525    0
526    0
527    0
528    0
529    0
Name: PASS, Length: 530, dtype: int64

In [26]:
#SMOTE uygulanmadan önce k=10 cross validation ile Decission Tree sonucu
c_dt = DecisionTreeClassifier(random_state=42)
np.mean(cross_val_score(c_dt, x, y, cv=10))


0.8832692307692307

In [27]:
#SMOTE uygulandıktan sonra k=10 cross validation ile Decission Tree sonucu
c_dt = DecisionTreeClassifier(random_state=42)
np.mean(cross_val_score(c_dt, X_res, y_res, cv=10))

0.9150943396226415

In [28]:
#SMOTE uygulanmadan önce k=10 cross validation ile Random Forrest sonucu
c_rf = RandomForestClassifier(random_state=42)
np.mean(cross_val_score(c_rf, x, y, cv=10))

0.9165384615384615

#SMOTE uygulandıktan sonra k=10 cross validation ile Random Forrest sonucu
c_rf = RandomForestClassifier(random_state=42)
np.mean(cross_val_score(c_rf, X_res, y_res, cv=10))

In [90]:
#Holdout yöntemi %70 eğitim %30 test SMOTE uygulanmamış
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=42)


In [91]:
#SMOTE uygulanmadan önce .30 holdout ile Decission Tree sonucu
c_dt = DecisionTreeClassifier(random_state=42)
c_dt.fit(X_train,y_train)
results = c_dt.predict(X_test)
accuracy_score(y_test, results)

0.865546218487395

In [92]:
#SMOTE uygulanmadan önce .30 holdout ile Random Forrest sonucu
c_rf = RandomForestClassifier(random_state=42)
c_rf.fit(X_train,y_train)
results = c_rf.predict(X_test)
accuracy_score(y_test, results)

0.8907563025210085

In [93]:
c_SVC = SVC(kernel='linear', random_state = 0)
c_SVC.fit(X_train, y_train)
c_SVC.score(X_train, y_train)

0.9891304347826086

In [94]:
c_AB = AdaBoostClassifier()
c_AB.fit(X_train, y_train)
c_AB.score(X_train,y_train)

0.9927536231884058

---------------SMOTE uygulandıktan sonra------------

In [85]:
#Holdout yöntemi %70 eğitim %30 test SMOTE uygulanmış
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.30, random_state=42)

In [86]:
#SMOTE uygulandıktan sonra .30 holdout ile Decission Tree sonucu
c_dt = DecisionTreeClassifier(random_state=42)
c_dt.fit(X_train,y_train)
results = c_dt.predict(X_test)
accuracy_score(y_test, results)

0.89937106918239

In [87]:
#SMOTE uygulandıktan sonra .30 holdout ile Random Forrest sonucu
c_rf = RandomForestClassifier(random_state=42)
c_rf.fit(X_train,y_train)
results = c_rf.predict(X_test)
accuracy_score(y_test, results)

0.9245283018867925

In [88]:
c_SVC = SVC(kernel='linear', random_state = 0)
c_SVC.fit(X_train, y_train)
c_SVC.score(X_train, y_train)

0.9811320754716981

In [89]:

c_AB = AdaBoostClassifier()
c_AB.fit(X_train, y_train)
c_AB.score(X_train,y_train)

0.9919137466307277